In [ ]:
!pip install shap==0.44.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install mrmr_selection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install minepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.0/497.0 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for minepy: filename=minepy-1.2.6-cp310-cp310-linux_x86_64.whl size=187102 sha256=c2c18acb1083fcca77e5d5f8f6df93fd84aeb3a625675f9e04b669b6914e6638
  Stored in directory: /root/.cache/pip/wheels/69/38/a6/825bb9b9ed81e6af43a0ef80c7cfe4cafcfdbc2f5cde2959d9
Successfully built minepy


In [ ]:
import pandas as pd
import numpy as np
import shap
import mrmr
from minepy import MINE
from sklearn.model_selection import train_test_split
from mrmr import mrmr_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ***mRmR feature selection***

In [ ]:
#df=pd.read_csv('/content/apple quality.csv',header=None)
df=pd.read_csv('/content/ColonTumor.csv',header=None)

X = df.iloc[:, :-1]
y = df.iloc[:, -1]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

S1 = pd.DataFrame()
k = 0
a1 = 0
acc1 = 0

for i in range(10):
 h = mrmr_classif(X=X_train, y=y_train, K=1)

 S1[i]=X_train[h]
 X_train.drop(h, axis=1, inplace=True)

 rf = RandomForestClassifier(n_estimators=50,random_state=42)
 X_train_selected = S1
 X_test_selected = X_test[S1.columns]
 rf.fit(X_train_selected, y_train)
 y_pred1 = rf.predict(X_test_selected)
 a1 = accuracy_score(y_test, y_pred1)
 if i == 1 or a1 > acc1:
      acc1 = a1
      S_prime = S1
'''rf.fit(S1, y_train)
 a1 = rf.score(S1, y_train)
 print("a1", a1)'''
'''y_pred = rf.predict(S1)
 a1 = (y_pred == y_train).mean()
 #accuracy = accuracy_score(y_train, y_pred)'''


'''if i == 1 or a1 > acc1:
      acc1 = a1
      S_prime = S1'''
print(S_prime)
print(acc1)

100%|██████████| 1/1 [00:00<00:00, 250.81it/s]

         0       1       2       3       4       5       6      7       8  \
3    138.0   165.0   209.0   338.0   196.0   334.0   442.0   95.2    43.3   
17   185.0   112.0   194.0   194.0   199.0    49.0    36.4   51.8   286.0   
48  1930.0   328.0   558.0   481.0   645.0   992.0  2640.0   83.9   422.0   
8   3890.0  1780.0  1850.0   598.0  1730.0   542.0  1250.0   54.2    28.9   
6   2030.0  1390.0  1450.0   387.0  1470.0   325.0   453.0   77.2    79.7   
33   287.0   265.0   436.0   147.0   443.0    43.6    59.0  147.0   327.0   
4    988.0   723.0   577.0   661.0   598.0   973.0  1330.0   36.3   393.0   
46   868.0   288.0   453.0   443.0   606.0   103.0   584.0  334.0  1280.0   
19   231.0   125.0   179.0   229.0   156.0    51.8    65.8  115.0   735.0   
53   660.0   184.0   393.0   217.0   393.0    76.7   608.0  115.0   835.0   
41   336.0    85.0   197.0    93.1   181.0   183.0   816.0   56.4   886.0   
54  3130.0   935.0  2310.0   581.0  2310.0  1340.0  4070.0   13.7   646.0   

# ***DFI***

In [ ]:
def calculate_feature_importance(S_prime):
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(S_prime, y_train)
    explainer = shap.TreeExplainer(rf_classifier)
    shap_values = explainer.shap_values(S_prime)
    #shap_values_array = np.array(shap_values)

    #print(shap_values.shape)

    f = np.sum(np.abs(shap_values), axis=0)
    #print(f.shape)

    avg = np.mean(f, axis=0)
    #print(avg.shape)
    return avg

def calculate_MIC(S_prime):
    mic_values = pd.DataFrame(index=S_prime.columns, columns=S_prime.columns)

    for col1 in S_prime.columns:
        for col2 in S_prime.columns:
            if col1 != col2:
                mine = MINE()
                mine.compute_score(S1[col1], S1[col2])
                mic_values.loc[col1, col2] = mine.mic()

    return mic_values

shapley = calculate_feature_importance(S_prime)
mic_values = calculate_MIC(S_prime)

def calculate_DFI(Shap, mic_values):
    DFI = shapley.copy()
    for col1 in S_prime.columns:
        for col2 in S_prime.columns:
            if col1 != col2:
                DFI[col1] *= (1 - mic_values.loc[col1, col2])
    return DFI
DFI=calculate_DFI(shapley,mic_values)
print("shap of each feature:", shapley)
print("mic values:\n", mic_values)
print("DFI of each feature is:", DFI)


shap of each feature: [0.14554125 0.07915555 0.02806992 0.15874618 0.02732766 0.04298763
 0.05108621 0.15872078 0.0526515  0.14945945]
mic values:
           0         1         2         3         4         5         6  \
0       NaN  0.786017  0.786907  0.689796  0.793088  0.617107  0.665645   
1  0.786017       NaN  0.852937  0.618108    0.8196  0.837217  0.620112   
2  0.786907  0.852937       NaN   0.56673  0.999673  0.754396  0.689796   
3  0.689796  0.618108   0.56673       NaN  0.645864  0.704417  0.521158   
4  0.793088    0.8196  0.999673  0.645864       NaN  0.718881  0.689796   
5  0.617107  0.837217  0.754396  0.704417  0.718881       NaN  0.788631   
6  0.665645  0.620112  0.689796  0.521158  0.689796  0.788631       NaN   
7  0.574885  0.409245  0.419983  0.333316  0.406047  0.272352  0.395411   
8  0.270951  0.370662  0.350045  0.311034  0.294411  0.379918  0.351964   
9  0.439825   0.41051  0.389531  0.382478  0.374692  0.389271  0.322084   

          7         8     

In [ ]:
DFI= pd.Series(DFI, dtype=float)


In [ ]:
decimal_values = [format(value, ".6f") for value in DFI]
print(decimal_values)

['0.000009', '0.000002', '0.000000', '0.000116', '0.000000', '0.000003', '0.000017', '0.000863', '0.000862', '0.000973']


In [ ]:
max(DFI)

1.5628926146127603e-250

In [ ]:
print(shapley)

[0.14554125 0.07915555 0.02806992 0.15874618 0.02732766 0.04298763
 0.05108621 0.15872078 0.0526515  0.14945945]


In [ ]:
S=0
def update_DFI(shap_values, mic_values, S_prime, S2):
    #DFI =  # Initialize an empty dictionary to store DFI values

    # Iterate over each feature in S1
    for col_s1 in S_prime.columns:
            DFI[col_s1] = shap_values[col_s1]
            # Initialize a variable to store the product of (1 - MIC) values
            product = 1.0
            # Iterate over each feature in S2
            for col_s2 in S2.columns:
                # Calculate the product of (1 - MIC) values
                product *= (1 - mic_values.loc[col_s1, col_s2])
            # Update DFI for the current feature in S1
            DFI[col_s1] = DFI[col_s1] *product
    return pd.Series(DFI, index=S_prime.columns)

S2 = pd.DataFrame()  # DataFrame to store selected features
k = 0
while k < 10 :
    DFI = pd.Series(DFI, index=S_prime.columns)
    # Select the feature with the largest DFI
    selected_feature = DFI.idxmax()
    # Add the selected feature to S2
    S2[selected_feature] = S_prime[selected_feature]  # Add the selected feature to S2
    S_prime.drop(selected_feature, axis=1, inplace=True)
    DFI = update_DFI(shapley, mic_values, S_prime, S2)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    X_train_selected = S2
    X_test_selected = X_test[S2.columns]
    rf.fit(X_train_selected, y_train)
    y_pred1 = rf.predict(X_test_selected)
    a2 = accuracy_score(y_test, y_pred1)
    #model.fit(S2, y_train)
    #a2 = model.score(S2, y_train)
    #y_pred = model.predict(S2)
    #a2 = np.mean(y_pred == y_train)
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")
    if a2 < acc1:
      S2.drop(selected_feature, axis=1, inplace=True)
    k += 1

print("S2 is: ",S2)

Iteration 1: Selected feature 9, Accuracy: 0.625
Iteration 2: Selected feature 3, Accuracy: 0.625
Iteration 3: Selected feature 7, Accuracy: 0.625
Iteration 4: Selected feature 0, Accuracy: 0.625
Iteration 5: Selected feature 8, Accuracy: 0.625
Iteration 6: Selected feature 6, Accuracy: 0.625
Iteration 7: Selected feature 1, Accuracy: 0.3125
Iteration 8: Selected feature 5, Accuracy: 0.625
Iteration 9: Selected feature 4, Accuracy: 0.625
Iteration 10: Selected feature 2, Accuracy: 0.625
S2 is:           9       3      7       0       8       6       5       4       2
3    233.0   338.0   95.2   138.0    43.3   442.0   334.0   196.0   209.0
17   525.0   194.0   51.8   185.0   286.0    36.4    49.0   199.0   194.0
48   374.0   481.0   83.9  1930.0   422.0  2640.0   992.0   645.0   558.0
8    132.0   598.0   54.2  3890.0    28.9  1250.0   542.0  1730.0  1850.0
6    148.0   387.0   77.2  2030.0    79.7   453.0   325.0  1470.0  1450.0
33  1050.0   147.0  147.0   287.0   327.0    59.0    43.

In [ ]:
if a2 < acc1 and k == 10:
    S = S_prime
else:
    S = S2
print("S is: ",S)

S is:           9       3      7       0       8       6       5       4       2
3    233.0   338.0   95.2   138.0    43.3   442.0   334.0   196.0   209.0
17   525.0   194.0   51.8   185.0   286.0    36.4    49.0   199.0   194.0
48   374.0   481.0   83.9  1930.0   422.0  2640.0   992.0   645.0   558.0
8    132.0   598.0   54.2  3890.0    28.9  1250.0   542.0  1730.0  1850.0
6    148.0   387.0   77.2  2030.0    79.7   453.0   325.0  1470.0  1450.0
33  1050.0   147.0  147.0   287.0   327.0    59.0    43.6   443.0   436.0
4    371.0   661.0   36.3   988.0   393.0  1330.0   973.0   598.0   577.0
46  1720.0   443.0  334.0   868.0  1280.0   584.0   103.0   606.0   453.0
19   378.0   229.0  115.0   231.0   735.0    65.8    51.8   156.0   179.0
53   734.0   217.0  115.0   660.0   835.0   608.0    76.7   393.0   393.0
41   420.0    93.1   56.4   336.0   886.0   816.0   183.0   181.0   197.0
54   184.0   581.0   13.7  3130.0   646.0  4070.0  1340.0  2310.0  2310.0
31   948.0   261.0  323.0   471

In [ ]:
S=0
'''def update_DFI(selected_feature, shap, mic_values, S1,DFI):
    for col in S1.index:
        if col != selected_feature:
            DFI[col] *= (1 - mic_values.loc[col, selected_feature])

    DFI[col] = shap[col] * np.prod(1 - mic_values.loc[col, selected_feature])

   # DFI[selected_feature] = shap[selected_feature] * np.prod(1 - mic_values.loc[:, selected_feature])

    return DFI'''
def update_DFI(shap_values, mic_values, S1, S2):
    #DFI =  # Initialize an empty dictionary to store DFI values

    # Iterate over each feature in S1
    for col_s1 in S1.columns:
            DFI[col_s1] = shap_values[col_s1]
            # Initialize a variable to store the product of (1 - MIC) values
            product = 1.0
            # Iterate over each feature in S2
            for col_s2 in S2.columns:
                # Calculate the product of (1 - MIC) values
                product *= (1 - mic_values.loc[col_s1, col_s2])
            # Update DFI for the current feature in S1
            DFI[col_s1] = DFI[col_s1] *product
    return pd.Series(DFI, index=S1.columns)


S2 = pd.DataFrame()  # DataFrame to store selected features
#DFI = pd.Series(av, index=S1.columns)  # Convert av to a pandas Series for compatibility
a2 = 0
#DFI = pd.Series(DFI, index=S1.columns)  # Initialize DFI with feature importances
k = 0  # Initialize loop variable
while k < 10 and a2 < acc1:
    DFI = pd.Series(DFI, index=S1.columns)
        # Select the feature with the largest DFI
    selected_feature = DFI.idxmax()

        # Add the selected feature to S2
    '''S2 = pd.concat([S2, pd.DataFrame(S1[selected_feature])], axis=1)
    #S2 = pd.concat([S2, pd.DataFrame(selected_feature)], axis=1)

    if selected_feature in S1.columns:
        selected_feature_data = S1[selected_feature]
        S2 = pd.concat([S2, pd.DataFrame(selected_feature_data)], axis=1)
        print(f"Selected feature {selected_feature} added to S2.")
    else:
        print(f"Warning: {selected_feature} not found in S1 columns.")'''
        # Remove the selected feature from S1
    '''S1.drop(selected_feature, axis=1, inplace=True)

            # Update DFI of remaining features
    DFI = update_DFI(shapley, mic_values, S1, S2)

    # Reinitialize the model inside the loop
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    # Reset indices for both S2 and y_train
    S2.reset_index(drop=True, inplace=True)
    y_train_reset = y_train.reset_index(drop=True)

    # Train a model with the updated feature set and calculate accuracy
    model.fit(S2, y_train_reset)
    #a2 = calculate_accuracy(model, S2, y_train_reset)
    y_pred = model.predict(S2)
    a2 = np.mean(y_pred == y_train)
    # Record accuracy and update loop variables
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")
    #print(DFI)
    #k += 1

    k += 1'''
    S2[selected_feature] = S1[selected_feature]  # Add the selected feature to S2
    S1.drop(selected_feature, axis=1, inplace=True)
    DFI = update_DFI(shapley, mic_values, S1, S2)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(S2, y_train)
    accuracy = model.score(S2, y_train)
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {accuracy}")
    k += 1
    if accuracy < acc1:
        break

    acc1 = accuracy
print(S2)

    #return S2, DFI
#K = 10

#selected_features, final_DFI = phase_2(S1.copy(), shapley, mic_values, y_train, K, acc1)

if a2 < acc1 and k == 10:
    S = S1
else:
    S = S2
#print(S)

ValueError: attempt to get argmax of an empty sequence

In [ ]:

def update_DFI(shap_values, mic_values, S1, S2):

    # Iterate over each feature in S1
    for col_s1 in S1.columns:
        DFI[col_s1] = shap_values[col_s1]
        # Initialize a variable to store the product of (1 - MIC) values
        product = 1.0
        # Iterate over each feature in S2
        for col_s2 in S2.columns:
            # Calculate the product of (1 - MIC) values
            product *= (1 - mic_values.loc[col_s1, col_s2])
        # Update DFI for the current feature in S1
        DFI[col_s1] *= product

    return pd.Series(DFI)


S2 = pd.DataFrame()  # DataFrame to store selected features
k = 0  # Initialize loop variable
acc1 = 0  # Initialize acc1

while k < 10 and acc1 < 0.9:
    DFI = pd.Series(DFI,index=S1.columns)  # Initialize DFI with feature importances
    selected_feature = DFI.idxmax()  # Select the feature with the largest DFI
    S2[selected_feature] = S1[selected_feature]  # Add the selected feature to S2
    S1.drop(selected_feature, axis=1, inplace=True)
    DFI = update_DFI(shapley, mic_values, S1, S2)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(S2, y_train)
    accuracy = model.score(S2, y_train)
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {accuracy}")
    k += 1

    if accuracy < acc1:
        break

    acc1 = accuracy

print(S2)


TypeError: reduction operation 'argmax' not allowed for this dtype

In [ ]:
'''import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def update_DFI(selected_feature, shap_values, mic_values, S1, S2):
    DFI = {}  # Initialize an empty dictionary to store DFI values

    # Iterate over each feature in S1
    for col_s1 in S1:
        if col_s1 != selected_feature:
            DFI[col_s1] = shap_values[col_s1]
            # Initialize a variable to store the product of (1 - MIC) values
            product = 1.0
            # Iterate over each feature in S2
            for col_s2 in S2:
                # Calculate the product of (1 - MIC) values
                product *= (1 - mic_values.loc[col_s1, col_s2])
            # Update DFI for the current feature in S1
            DFI[col_s1] *= product

    return DFI


DFI = pd.Series(shapley)  # Initialize DFI with shapley values

k = 0  # Initialize loop variable
acc1 = 0.8  # Set the desired accuracy threshold
a2 = 0  # Initialize accuracy variable

while k < 10 and a2 < acc1:
    # Select the feature with the largest DFI
    selected_feature = DFI.idxmax()

    # Add the selected feature to S2
    if selected_feature in S1.columns:
        selected_feature_data = S1[selected_feature]
        S2 = pd.concat([S2, pd.DataFrame(selected_feature_data)], axis=1)
        print(f"Selected feature {selected_feature} added to S2.")
    else:
        print(f"Warning: {selected_feature} not found in S1 columns.")

    # Remove the selected feature from S1
    S1.drop(selected_feature, axis=1, inplace=True)

    # Update DFI of remaining features
    DFI = update_DFI(selected_feature, shapley, mic_values, S1, S2)

    # Reinitialize the model inside the loop
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train a model with the updated feature set and calculate accuracy
    model.fit(S2, y_train)
    y_pred = model.predict(S2)
    a2 = accuracy_score(y_train, y_pred)

    # Record accuracy and update loop variables
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")

    k += 1

print(S2)'''


KeyError: '[3] not found in axis'

In [ ]:
'''import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

def update_DFI(selected_feature, shapley, mic_values, S1, S2):
    DFI = {}  # Initialize an empty dictionary to store DFI values

    # Iterate over each feature in S1
    for col_s1 in S1:
        if col_s1 != selected_feature:
            DFI[col_s1] = shapley[col_s1]
            # Initialize a variable to store the product of (1 - MIC) values
            product = 1.0
            # Iterate over each feature in S2
            for col_s2 in S2:
                # Calculate the product of (1 - MIC) values
                product *= (1 - mic_values.loc[col_s1, col_s2])
            # Update DFI for the current feature in S1
            DFI[col_s1] *= product

    return pd.Series(DFI)

DFI = pd.Series(DFI, index=S1.columns)
k = 0  # Initialize loop variable
acc1 = 0.8  # Set the desired accuracy threshold
a2 = 0  # Initialize accuracy variable

while k < 10 and a2 < acc1:

    # Select the feature with the largest DFI in S1
    selected_feature = DFI.idxmax()

    # Add the selected feature to S2
    if selected_feature in S1.columns:
        selected_feature_data = S1[selected_feature]
        S2 = pd.concat([S2, pd.DataFrame(selected_feature_data)], axis=1)
        print(f"Selected feature {selected_feature} added to S2.")
    else:
        print(f"Warning: {selected_feature} not found in S1 columns.")

    # Remove the selected feature from S1
    S1.drop(selected_feature, axis=1, inplace=True)
    DFI = update_DFI(None, shapley, mic_values, S1, S2)  # Update DFI values

    # Reinitialize the model inside the loop
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    S2.reset_index(drop=True, inplace=True)
    y_train_reset = y_train.reset_index(drop=True)
    # Train a model with the updated feature set and calculate accuracy
    model.fit(S2, y_train_reset)
    y_pred = model.predict(S2)
    a2 = np.mean(y_pred == y_train_reset)

    # Record accuracy and update loop variables
    #print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")
    print(selected_feature)
    print(a2)

    k += 1

print(S2)'''


Selected feature 6 added to S2.


ValueError: Input X contains NaN.
RandomForestClassifier does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X_train_selected = S
X_test_selected = X_test[S.columns]

rf1 = RandomForestClassifier(n_estimators=50,random_state=42)
rf1.fit(X_train_selected, y_train)

y_pred1 = rf1.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred1)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf1, X_train_selected, y_train, cv=2)
print('Mean Accuracy:', scores.mean())

'''classification_rep = classification_report(y_test, y_pred1)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report using Random Forest Classifier:")
print(classification_rep)'''


Mean Accuracy: 0.8940217391304348


'classification_rep = classification_report(y_test, y_pred1)\n\nprint(f"Accuracy: {accuracy:.4f}")\nprint("Classification Report using Random Forest Classifier:")\nprint(classification_rep)'

# ***Update DFI***

In [ ]:
'''def update_DFI(selected_feature, av, mic_values, DFI):
    for col in DFI.index:
        if col != selected_feature:
            DFI[col] *= (1 - mic_values.loc[col, selected_feature])
    DFI *= av[selected_feature]
    return DFI'''
def update_DFI(selected_feature, shapley, mic_values, DFI):
    for col in DFI.index:
        if col != selected_feature:
            DFI[col] *= (1 - mic_values.loc[col, selected_feature])
    DFI[col] = av[col] * np.prod(1 - mic_values.loc[col, selected_feature])
    return DFI

'''def calculate_accuracy(model, X_train, y_train):
    y_pred = model.predict(X_train)
    accuracy = np.mean(y_pred == y_train)
    return accuracy'''

S2 = pd.DataFrame()
a2=0
#DFI = pd.Series(shapley, index=S1.columns)
k = 0

#def phase_2(S1, shapley, mic_values, y_train, K, acc):

S2 = pd.DataFrame()  # DataFrame to store selected features
    #DFI = pd.Series(av, index=S1.columns)  # Convert av to a pandas Series for compatibility
a2=0

DFI = pd.Series(av, index=S1.columns)  # Initialize DFI with feature importances

k = 0  # Initialize loop variable


while k < 10 and a2 < acc1:  # Use K instead of k
        # Select the feature with the largest DFI
    selected_feature = DFI.idxmax()
    print(selected_feature)
        # Add the selected feature to S2
    S2 = pd.concat([S2, pd.DataFrame(S1[selected_feature])], axis=1)
    print(S2)
       # print("S2", S2)
        # Remove the selected feature from S1
    S1.drop(selected_feature, axis=1, inplace=True)

        # Update DFI of remaining features
    DFI = update_DFI(selected_feature, shapley, mic_values, DFI)

        # Reinitialize the model inside the loop
    model = RandomForestClassifier(n_estimators=100, random_state=42)

        # Reset indices for both S2 and y_train
    S2.reset_index(drop=True, inplace=True)
    y_train_reset = y_train.reset_index(drop=True)

        # Train a model with the updated feature set and calculate accuracy
    model.fit(S2, y_train_reset)
        #a2 = calculate_accuracy(model, S2, y_train_reset)
    y_pred = model.predict(S2)
    a2 = np.mean(y_pred == y_train)

        # Record accuracy and update loop variables
    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")
    k += 1

      #return S2, DFI
#K=10
#selected_features, final_DFI = phase_2(S1.copy(), shapley, mic_values, y_train, K, acc1)

'''if a2 < acc1 and k == K:
    selected_features = S1
else:
    selected_features = S2

print("\nSelected Features:")
print(selected_features)'''

ValueError: Length of values (10) does not match length of index (9)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import shap
from minepy import MINE

# Function to calculate feature importance using Shapley values
def calculate_feature_importance(S1):
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(S1, y_train)
    explainer = shap.Explainer(rf_classifier)
    shap_values = explainer.shap_values(S1)
    f = np.sum(np.abs(shap_values), axis=0)
    av = np.mean(f, axis=0)
    return av

# Function to calculate MIC values between features
def calculate_MIC(S1):
    mic_values = pd.DataFrame(index=S1.columns, columns=S1.columns)

    for col1 in S1.columns:
        for col2 in S1.columns:
            if col1 != col2:
                mine = MINE()
                mine.compute_score(S1[col1], S1[col2])
                mic_values.loc[col1, col2] = mine.mic()

    return mic_values

# Function to update DFI values for the remaining features
def update_DFI(selected_feature, av, mic_values, DFI):
    for col in DFI.index:
        if col != selected_feature:
            # Update based on MIC values with respect to selected_feature
            DFI[col] *= (1 - mic_values.loc[col, selected_feature])
    # Update DFI for the selected feature
    DFI[selected_feature] = av[selected_feature] * np.prod(1 - mic_values.loc[selected_feature, DFI.index])
    return DFI

# Function to perform the feature selection process
def feature_selection(S1, y_train, K):
    S2 = pd.DataFrame()  # DataFrame to store selected features
    av = calculate_feature_importance(S1)
    mic_values = calculate_MIC(S1)
    DFI = pd.Series(av, index=S1.columns)  # Initialize DFI with feature importances
    k = 0  # Initialize loop variable

    while k < K:
        # Select the feature with the highest DFI
        selected_feature = DFI.idxmax()

        # Add the selected feature to S2
        S2 = pd.concat([S2, pd.DataFrame(S1[selected_feature])], axis=1)

        # Remove the selected feature from S1
        S1.drop(selected_feature, axis=1, inplace=True)

        # Update DFI of remaining features
        DFI = update_DFI(selected_feature, av, mic_values, DFI)

        # Reinitialize the model inside the loop
        model = RandomForestClassifier(n_estimators=100, random_state=42)

        # Reset indices for both S2 and y_train
        S2.reset_index(drop=True, inplace=True)
        y_train_reset = y_train.reset_index(drop=True)

        # Train a model with the updated feature set and calculate accuracy
        model.fit(S2, y_train_reset)
        y_pred = model.predict(S2)
        a2 = np.mean(y_pred == y_train)

        # Record accuracy and update loop variables
        print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {a2}")
        k += 1

    return S2

# Example usage
K = 3  # Number of iterations
y_train = pd.Series([0, 1, 0, 1, 1, 0, 1, 0, 1, 0])  # Replace this with your actual y_train
S1 = pd.DataFrame(np.random.rand(10, 5), columns=['f1', 'f2', 'f3', 'f4', 'f5'])  # Replace this with your actual S1

selected_features = feature_selection(S1.copy(), y_train, K)
print("Selected Features:", selected_features.columns)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
X_train_selected = selected_feature
X_test_selected = X_test[selected_feature.columns]

rf1 = RandomForestClassifier(n_estimators=50,random_state=42)
rf1.fit(X_train_selected, y_train)

y_pred1 = rf1.predict(X_test_selected)

accuracy = accuracy_score(y_test, y_pred1)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rf1, X_train_selected, y_train, cv=2)
print('Mean Accuracy:', scores.mean())

classification_rep = classification_report(y_test, y_pred1)

print(f"Accuracy: {accuracy:.4f}")
print("Classification Report using Random Forest Classifier:")
print(classification_rep)

NameError: name 'selected_feature' is not defined

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from minepy import MINE
import shap

# Function to calculate feature importance using Shapley values
def calculate_feature_importance(S1, y_train):
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(S1, y_train)

    explainer = shap.Explainer(rf_classifier)
    shap_values = explainer.shap_values(S1)
    f = np.sum(np.abs(shap_values), axis=0)
    av = np.mean(f, axis=0)
    return av

# Function to calculate MIC values
def calculate_MIC(S1):
    mic_values = pd.DataFrame(index=S1.columns, columns=S1.columns)

    for col1 in S1.columns:
        for col2 in S1.columns:
            if col1 != col2:
                mine = MINE()
                mine.compute_score(S1[col1], S1[col2])
                mic_values.loc[col1, col2] = mine.mic()

    return mic_values

# Function to calculate DFI (Shapley * Product of (1 - MIC))
def calculate_DFI(S1, av, mic_values):
    DFI = av.copy()
    for col1 in S1.columns:
        for col2 in S1.columns:
            if col1 != col2:
                DFI[col1] *= (1 - mic_values.loc[col1, col2])

    return DFI

# Function to update DFI based on the selected feature in S2
def update_DFI(selected_feature, DFI, av, mic_values):
    for col in DFI.index:
        if col != selected_feature:
            DFI[col] *= (1 - mic_values.loc[col, selected_feature])
    DFI *= av  # Update DFI using the feature importance (shap values)

    return DFI

# Function to calculate accuracy
def calculate_accuracy(model, X, y):
    y_pred = model.predict(X)
    accuracy = np.mean(y_pred == y)
    return accuracy

# Sample data
data = {
    'f1': [1, 2, 3, 4, 5],
    'f2': [2, 3, 4, 5, 6],
    'f3': [3, 4, 5, 6, 7],
    'f4': [4, 5, 6, 7, 8],
}

df = pd.DataFrame(data)
y_train = pd.Series([0, 1, 0, 1, 0])

# Initialize S2 and DFI
S1 = df.copy()
S2 = pd.DataFrame()
DFI = pd.Series(index=df.columns)

# Set the desired number of iterations
K = 2

# Initialize loop variables
k = 0
acc1 = 0

while k < K:
    # Calculate feature importance using Shapley values
    av = calculate_feature_importance(S1, y_train)

    # Calculate MIC values
    mic_values = calculate_MIC(S1)

    # Calculate DFI
    DFI = calculate_DFI(S1, av, mic_values)

    # Select the feature with the highest DFI
    selected_feature = DFI.idxmax()

    # Add the selected feature to S2
    S2[selected_feature] = S1[selected_feature]

    # Remove the selected feature from S1
    S1.drop(selected_feature, axis=1, inplace=True)

    # Update DFI of remaining features
    DFI = update_DFI(selected_feature, DFI, av, mic_values)

    # Reinitialize the model inside the loop
    model = RandomForestClassifier(n_estimators=100, random_state=42)

    # Train a model with the updated feature set and calculate accuracy
    model.fit(S2, y_train)
    accuracy = calculate_accuracy(model, S2, y_train)

    print(f"Iteration {k + 1}: Selected feature {selected_feature}, Accuracy: {accuracy}")

    if accuracy < acc1:
        break

    k += 1

# Check if S2 or S1' should be selected based on accuracy
if accuracy < acc1 and k == K:
    selected_features = S1
else:
    selected_features = S2

print("\nSelected Features:")
print(selected_features)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices